# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/fernando/spark-2.4.6-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregproject').getOrCreate()

In [2]:
data = spark.read.csv('Logistic_Regression/customer_churn.csv', inferSchema=True, header=True)

In [3]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

## Select useful features

In [13]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=[
    'Age',
    'Total_Purchase', 
    'Account_Manager', 
    'Years', 
    'Num_Sites'], outputCol='features')

In [16]:
output = assembler.transform(data)

In [17]:
final_data = output.select(['features', 'churn'])

## Split and train

In [18]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [19]:
from pyspark.ml.classification import LogisticRegression

In [20]:
lr_churn = LogisticRegression(labelCol='churn')

## Fit model to data

In [21]:
fitted_churn_model = lr_churn.fit(train_churn)

In [22]:
training_sum = fitted_churn_model.summary

In [23]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                633|                633|
|   mean|0.15955766192733017|0.11690363349131122|
| stddev| 0.3664849197886583| 0.3215595184824268|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



## Evaluation Metrics

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [26]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.79276008254714...|[0.99177860567931...|       0.0|
|[25.0,9672.03,0.0...|    0|[4.62811677643124...|[0.99032144315366...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.44628329401367...|[0.99570621360254...|       0.0|
|[28.0,11128.95,1....|    0|[4.31073650824230...|[0.98675414848131...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.36420437536897...|[0.98743511010701...|       0.0|
|[29.0,10203.18,1....|    0|[3.87857527027049...|[0.97973874028548...|       0.0|
|[29.0,12711.15,0....|    0|[5.31219167775136...|[0.99509308903795...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.26823038069117...|[0.96332269863824...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.88587553377296...|[0.99722928562157...|       0.0|
|[31.0,7073.61,0

## Check area under the curve

In [27]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [28]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [29]:
auc

0.7788335517693316

## Try to predict on new data

In [30]:
final_lr_model = lr_churn.fit(final_data)

In [31]:
new_customers = spark.read.csv('Logistic_Regression/new_customers.csv', inferSchema=True, header=True)

In [32]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [33]:
test_new_customers = assembler.transform(new_customers)

In [34]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [35]:
final_results = final_lr_model.transform(test_new_customers)

In [40]:
final_results.select(['Company', 'prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

